In [3]:

from __future__ import (unicode_literals, absolute_import, division,
                        print_function)
import six

import os
from collections import OrderedDict

def read_log_from_script(path_to_log):
    """
    Parse the log that is output from the `dev-build` script

    Parameters
    ----------
    path_to_log : str
        The path to the log file that results from `bash dev-build > log 2>&1`

    Yields
    ------
    package : str
        The name of the package that is being built
    output : list
        The lines that were output for the build/test/upload of `package`
    """
    BUILD_START_LINE = '/tmp/staged-recipes'
    PACKAGE_NAME_LINE = '# $ anaconda upload '
    full_path = os.path.abspath(path_to_log)
    output = []
    package_name = ''
    with open(full_path, 'r') as f:
        for line in f.readlines():
            # remove white space and newline characters
            line = line.strip()
            if line.startswith(PACKAGE_NAME_LINE):
                # split the line on the whitespace that looks something like:
                # "# $ anaconda upload /tmp/root/ramdisk/mc/conda-bld/linux-64/album-v0.0.2_py35.tar.bz2"
                built_package_path = line.split()[-1]
                # remove the folder path
                built_package_name = os.path.split(built_package_path)[-1]
                # trim the '.tar.bz2'
                built_name = built_package_name[:-8]
            if line.startswith(BUILD_START_LINE):
                # always have to treat the first package differently...
                if package_name != '':
                    yield package_name, built_name, output
                package_name = os.path.split(line)[1]
                built_name = '%s-build-name-not-found' % package_name
                output = []
            else:
                output.append(line)
    
    yield package_name, built_name, output

In [4]:
def parse_conda_build(lines_iterable):
    """
    Group the output from conda-build into
    - 'build_init'
    - 'build'
    - 'test'
    - 'upload'
    """
    from collections import defaultdict
    bundle = []
    next_line_might_be_test = False
    init = True
    build = False
    test = False
    key = 'init'
    ret = []
    for line in lines_iterable:
        bundle.append(line)
        # init
        if init:
            if line.startswith("BUILD START"):
                line = bundle.pop()
                ret.append((key, bundle))
                bundle = [line]
                init = False
                build = True
                key = 'build'
        # build
        if build:
            if line.startswith("BUILD END"):
                next_line_might_be_test = True
                build = False
                continue
        # determine if test or upload comes next
        if next_line_might_be_test:
            next_line_might_be_test = False
            line = bundle.pop()
            ret.append((key, bundle))
            if line.startswith("TEST START"):
                test = True
                key = 'test'
                bundle = [line]
            elif line.startswith('Nothing to test for'):
                ret.append(('test', [line]))
                bundle = []
                key = 'upload'
            else:
                key = 'upload'
                bundle = [line]
        # test
        if test:
            if line.startswith("TEST END"):
                ret.append((key, bundle))
                bundle = []
                test = False
                key='upload'
    
    if bundle:
        ret.append((key, bundle))
    return OrderedDict(ret)

In [5]:
log = 'build.log'
gen = list(read_log_from_script(log))
parsed = {built_name: parse_conda_build(lines) for name, built_name, lines in gen}
width = max([len(name) for name in parsed.keys()])
for name, groups in sorted(parsed.items()):
    print(('{:%ds} -- {}' % width).format(name, [key for key, bundle in groups.items()]))

album-0.0.2-py35_0                                 -- ['init', 'build', 'test', 'upload']
album-v0.0.2.post0-0_g6b05c00_py35                 -- ['init', 'build', 'test', 'upload']
amx_configuration-2-1                              -- ['init', 'build', 'test', 'upload']
analysis-2015_03-py35_2                            -- ['init', 'build', 'test', 'upload']
args-0.1.0-py35_0                                  -- ['init', 'build', 'test', 'upload']
autoconf-2.69-1                                    -- ['init', 'build', 'test', 'upload']
automake-1.14-2                                    -- ['init', 'build', 'test', 'upload']
bluesky-0.3.1-1_py35                               -- ['init', 'build', 'test', 'upload']
bluesky-v0.4.0rc1.post105-105_g618d456_py35        -- ['init', 'build', 'test', 'upload']
boltons-15.0.0-py35_0                              -- ['init', 'build', 'test', 'upload']
boltons-16.0.0.post6-6_g968841a                    -- ['init', 'build', 'test', 'upload']
channelarc

In [64]:
def parse_init(init_section):
    ret = {}
    gen = (line for line in init_section)
    ret['err'] = []
    for line in gen:
        if 'CONDA_CMD' in line:
            ret['build_command'] = line.split('-->')[1].strip()
        line, err = check_for_errors(line, gen)
        if err:
            ret['err'].append(err)
    return ret

In [67]:
parse_init(parsed['tifffile-build-name-not-found']['init'])

{'build_command': 'conda-build /tmp/staged-recipes/recipes/tifffile --python=3.5',
 'err': [["Error: 'numpy x.x' requires external setting"]]}

In [7]:
parse_init(parsed['readline-build-name-not-found']['init'])

{'build_command': 'conda-build /tmp/staged-recipes/recipes/readline --python=3.5'}

In [50]:
def check_for_errors(line, gen):
    ERROR = "Error: "
    TRACEBACK = 'Traceback (most recent call last):'
    err = []
    if line.startswith(ERROR) or line == TRACEBACK:
        if line.startswith(ERROR) or line == TRACEBACK:
            try:
                while line != '':
                    err.append(line)
                    line = next(gen)
            except StopIteration:
                # this is thrown when the error goes to the end of the 
                # log section
                pass
    return line, err

In [51]:
def parse_build(build_section):
    gen = (line for line in build_section)
    PACKAGE_NAME = 'Package: '
    ret = {'error': []}
    error = False
    traceback = False
    lines = []
    for line in gen:
        if PACKAGE_NAME in line:
            # format the package name
            ret['built_name'] = line[len(PACKAGE_NAME):]
        line, err = check_for_errors(line, gen)
        if err:
            ret['error'].append(err)
            ret['built_name'] = 'failed'
    return ret

In [52]:
ret = parse_build(parsed['pymongo-build-name-not-found']['build'])

ret

{'built_name': 'failed',
 'error': [['Error: Connection error: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:645): https://pypi.python.org/packages/source/p/pymongo/pymongo-2.9.1.tar.gz'],
  ['Error: Could not download pymongo-2.9.1.tar.gz']]}

In [56]:
p = {pkg_name: parse_build(grouped['build']) for pkg_name, grouped in parsed.items() if 'build' in grouped}

In [57]:
for pkg_name, parsed_build in p.items():
    if parsed_build['built_name'] == 'failed':
        print(pkg_name)

super_state_machine-build-name-not-found
keyring-build-name-not-found
humanize-build-name-not-found
hgtools-build-name-not-found
readline-build-name-not-found
slicerator-build-name-not-found
tzlocal-build-name-not-found
pymongo-build-name-not-found


In [15]:
def parse_test(test_section):
    ret = {'error': {}}
    for line in test_section:
        print(line)

In [19]:
# ophyd has an error, let's investigate that a little more
parse_test(parsed['automake-1.14-2']['test'])

TEST START: automake-1.14-2
Fetching package metadata: ........
Solving package specifications: ....+ automake --help
Usage: /tmp/root/ramdisk/mc/envs/_test/bin/automake [OPTION]... [Makefile]...

Generate Makefile.in for configure from Makefile.am.

Operation modes:
--help               print this help, then exit
--version            print version number, then exit
-v, --verbose            verbosely list files processed
--no-force           only update Makefile.in's that are out of date
-W, --warnings=CATEGORY  report the warnings falling in CATEGORY

Dependency tracking:
-i, --ignore-deps      disable dependency tracking code
--include-deps     enable dependency tracking code

Flavors:
--foreign          set strictness to foreign
--gnits            set strictness to gnits
--gnu              set strictness to gnu

Library files:
-a, --add-missing      add missing standard files to package
--libdir=DIR       set directory storing library files
--print-libdir     print directory storing